# Collection from Chinese Science Bulletin - hard scientific articles

NOT CLEANED YET AS OF 21 OCT 2024

BUT UPLOADED AS THIS WILL HAPPEN SOON (and uploading creates pressure to do so!)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
import itertools
import time
import random
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
options.add_argument("--disable-infobars")
#options.add_argument("--disable-extensions")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

###################################
#options.add_argument("--headless")
##### Does not work in headless!!!! #####
#prefs = {"profile.managed_default_content_settings.images":1} #2
#options.add_experimental_option("prefs", prefs)

##### This may work in headless #####
#options.add_argument('--blink-settings=imagesEnabled=true')
####################################

options.add_argument("user-data-dir=selenium2")

# Get URLs

In [ ]:
driver = webdriver.Chrome(options=options) 


In [ ]:
base_url = 'https://www.sciengine.com/publisher/scp/journal/CSB/'
close = '?slug=browse'

In [ ]:
# Assume I got these through recon, will have to check and then improve to make this more automatic
# See https://www.sciengine.com/CSB/catalogue 
# could search that to allow year (volumn) with 3 issues per month (1-36, so could also collect by month)
# For me that worked, but for most people/use cases not relevant, would want keyword searches

volumns =  list(range(62, 68))
issues = list(range(1, 37))

In [ ]:
vi = list(itertools.product(volumns, issues))

In [ ]:
url_list = []
redo_list = []

In [ ]:
for i in vi:
    full_url = base_url + str(i[0]) + '/' + str(i[1]) + close
    url_list.append(full_url)

In [ ]:
redo_list = []
doi_list = []

for url in url_list:
    try:
        driver.get(url)
        print(url)
        driver.implicitly_wait(10)
        content = driver.find_elements(By.PARTIAL_LINK_TEXT,"doi")
        print('Content found...')

        for c in content:
            doi_list.append(c.text)
        
        nap_time = random.uniform(1, 2)
        print('Waiting: ' + str(nap_time))
        time.sleep(nap_time)
        
    except:
        print('Error')
        redo_list.append(url)

In [ ]:
retry = ['https://www.sciengine.com/publisher/scp/journal/CSB/66/28-29?slug=browse', 
'https://www.sciengine.com/publisher/scp/journal/CSB/66/4-5?slug=browse', 
'https://www.sciengine.com/publisher/scp/journal/CSB/65/28-29?slug=browse', 
'https://www.sciengine.com/publisher/scp/journal/CSB/65/2-3?slug=browse', 
'https://www.sciengine.com/publisher/scp/journal/CSB/64/28-29?slug=browse', 
'https://www.sciengine.com/publisher/scp/journal/CSB/64/5-6?slug=browse']

In [ ]:
for url in retry:
    try:
        driver.get(url)
        print(url)
        driver.implicitly_wait(10)
        content = driver.find_elements(By.PARTIAL_LINK_TEXT,"doi")
        print('Content found...')

        for c in content:
            doi_list.append(c.text)
        
        nap_time = random.uniform(1, 2)
        print('Waiting: ' + str(nap_time))
        time.sleep(nap_time)
        
    except:
        '''
        Did not work due to re-directs.
        '''
        print('Error')
        redo_list.append(url)

In [ ]:
len(doi_list)

In [ ]:
df = pd.DataFrame(doi_list, columns=['url'])

In [ ]:
df = df.drop_duplicates()
print(len(df))

# Gets allot of text info if needed
# new = driver.find_elements_by_class_name('dihuias')
# for n in new:
#     print(n.text)

In [ ]:
df1 = df[~df['url'].str.contains('csb')]
#Alternative
#df1 = df[df['url'].str.len() < 41]

In [ ]:
len(df1)

In [ ]:
PATH = './DATA/URLS/'

In [ ]:
df1.to_csv(PATH + 'csb_urls.csv', index = False)

# Load here 

In [ ]:
df = pd.read_csv(PATH + 'csb_urls.csv')

# Requests

In [ ]:
import requests
#import bs4
#import json
import pandas as pd
import bs4
#import csv
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import ssl
import urllib.request
from fake_useragent import UserAgent
import time
import random

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
ua = UserAgent(verify_ssl=False, cache=False)
user_agent = ua.random
headers = {'User-Agent': user_agent}

In [ ]:
def make_request(url):
    page = urllib.request.Request(url, headers=headers, allow_redirects=False) 
    time.sleep(5)
    uClient = uReq(page)
    time.sleep(2)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, 'lxml')
    return page_soup

In [ ]:
def get_details(page_soup):
    time.sleep(1)
    result_list = page_soup.find('div', class_= 'article_title')
    url_title = result_list.find_all('a') #('a', {'class': 'dihuias'})
    dates = result_list.find_all('span')
    #urls = page_soup.find_all('li') #('a', {'class': 'dihuias'})
    for u in url_title:
        item = u['href']
        item = base_url + item
        url_list.append(item)
    for t in url_title:
        item = t.text
        title_list.append(item)
    for d in dates:
        item_d = d.text
        date_list.append(item_d)

In [ ]:
def connect(index, url):

    print('Fetching ' + str(index))
    page_soup = make_request(url)        
    
    nap_time = random.uniform(.2, 1.5)
    print('Waiting: ' + str(nap_time))
    time.sleep(nap_time)

    text = page_soup.find('div', id='content')
    text_list.append(text)
    #counter +=1

In [ ]:
counter = 1
for index, row in df1.iterrows():
    try:
        connect(index, row.url)
    except:
        print('Connection error...sleeping for 2 minutes')
        time.sleep(120)
        print('Retrying...')
        connect(index, row.url)
print(len(text_list))

In [ ]:
#page_soup = make_request('https://doi.org/10.1360/N972016-00736')

In [ ]:
r = requests.head('https://doi.org/10.1360/N972016-00736', allow_redirects=False)

In [ ]:

r.history

In [ ]:
import requests

def expand_url(url):
  s = requests.Session()
  #s.allow_redirects = -1
  try:
     r = s.get(url.rstrip(),allow_redirects=3,verify=False)
     print([resp.url for resp in r.history])
     return r.url.rstrip()
  except requests.exceptions.ConnectionError as e:
    print(e)


In [ ]:
#expand_url('https://doi.org/10.1360/N972016-00736')

In [ ]:
title = page_soup.find_all('//*', id='abstractText')
#//*[@id="abstractText"]
for t in title:
    print(t)

In [ ]:
title = page_soup.find('//div', id='fulltextContent')

print(title.text)


### Selenium

In [ ]:
test_url = 'https://doi.org/10.1360/N972016-00736'

In [ ]:
df.tail(2)

In [ ]:
df.url[0]

# Tests

In [ ]:
driver = webdriver.Chrome(options=options) 

In [ ]:
driver.get(test_url)

In [ ]:
driver.refresh()

In [ ]:

#content = driver.find_element(By.XPATH,"//div[@id='fulltextContent']")
content = get_content()

#for c in content:
content_list.append(content)

In [ ]:
len(content_list)

In [ ]:
content = driver.find_element(By.XPATH,"//div[@id='fulltextContent']")

content2 = content.find_elements(By.CSS_SELECTOR,"p:not([style='center'])") #[id='fulltextContent']") #[content-type='dispformula']
for c in content2:
    print(c.text)

In [ ]:
content = driver.find_element(By.XPATH,"//div[@id='fulltextContent']")
#aras = content.find_elements(By.XPATH, 'p') 

table = content.find_elements(By.CSS_SELECTOR,"p:not([table-wrap])")
for t in table:
    print(t.text)
#print(content)


# Main Loop

In [ ]:
content_list = []
title_list = []
date_list = []
url_list = []
fail_list = []

In [ ]:
def get_meta():
    title = driver.find_element(By.XPATH, "//*[@id='articletitle']")
    print('Title found...')  
    
    date = driver.find_element(By.XPATH, "//ul[@class='xuemaoli']")
    print('Date found...')

    return title, date
    


In [ ]:
def get_content():
    try:
        #WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//div[@id='sec-001']")))#.get_attribute("value")
        content = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH,"//div[@id='fulltextContent']")))
        print('Content found...' + str(len(content.text)))
        title, text = get_meta()

    except:
        driver.refresh()
        get_content()
        
    if len(content.text) == 0:
        driver.refresh()
        driver.implicitly_wait(3)
        time.sleep(5)
        get_content()
        #content = driver.find_element(By.XPATH,"//div[@id='fulltextContent']")

    return content.text, title, text

In [ ]:
def connect(index, url):
    
    
    driver.get(url)
    print(index)
    print(url)
    
    #driver.implicitly_wait(5)
    #driver.set_page_load_timeout(20)
    
    #content, title, text = get_content()
    nap_time = random.uniform(2, 3)
    
    time.sleep(nap_time)
    
    try:
        #content = WebDriverWait(driver, 30).until(EC.text_to_be_present_in_element_attribute((By.XPATH,"//div[@id='fulltextContent']")))
        WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR,"p")))
        #print('Loaded')
        check = driver.find_element(By.CSS_SELECTOR,"h2")
        print(check.text)
        if check.text == 'DOI Not Found':
            print('Not found: passing...')
            pass
        
        content = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH,"//div[@id='fulltextContent']")))
        #content = driver.find_element(By.XPATH,"//div[@id='fulltextContent']")
        
        while len(content.text) == 0 or content.text is None:
            print('ZERO')
            #driver.refresh()
            #driver.implicitly_wait(3)
            time.sleep(nap_time)
            # time.sleep(5)
            #connect(index, url)
            content = WebDriverWait(driver, 30).until(EC.text_to_be_present_in_element_attribute((By.XPATH,"//div[@id='fulltextContent']")))

        #else:
        print('Content found...' + str(len(content.text)))

        title = driver.find_element(By.XPATH, "//*[@id='articletitle']")
        print('Title found...')  
        
        date = driver.find_element(By.XPATH, "//ul[@class='xuemaoli']")
        print('Date found...')

        
        #for c in content:
        content_list.append(content.text)
        #for t in title:
        title_list.append(title.text)       
        #for d in date:
        date_list.append(date.text)       
        
        url_list.append(url)
        
        nap_time = random.uniform(4, 6)
        print('Waiting: ' + str(nap_time))
        time.sleep(nap_time)
    
    except:
        #######NOT YET FUNCTIONAL!
        print('Failed')
        fail_list.append(url)
        pass
        #get_content()


In [ ]:
df_slice = df[1008:] # last line -1
df_slice.head(5)

In [ ]:
issue = 'https://doi.org/10.1360/N972018-01255'

In [ ]:
urls = []
for index, row in df_slice.iterrows():
    driver.get(row.url)
    driver.implicitly_wait(20)
    driver.set_page_load_timeout(20)

    url = driver.current_url
    print(url)
    urls.append(url)


In [ ]:
driver = webdriver.Chrome(options=options)

In [ ]:
for index, row in df_slice.iterrows():
    try:
        connect(index, row.url)
        print(str(len(date_list)) + ' - ' + str(len(title_list)) + ' - ' + str(len(content_list)) + ' - ' + str(len(url_list)))
    except NoSuchElementException:
        print('Failed...')
        fail_list.append(row.url)
        pass
    except:
        print('Connection error...sleeping for 2 minutes')
        time.sleep(120)
        print('Retrying...')
        connect(index, row.url)

In [ ]:
print(str(len(date_list)) + ' - ' + str(len(title_list)) + ' - ' + str(len(content_list)) + ' - ' + str(len(url_list)))

In [ ]:
df2 = pd.DataFrame(list(zip(title_list, date_list, content_list, url_list)), columns =['title', 'date', 'text', 'url']) 

In [ ]:
df2.tail(3)

In [ ]:
PATH_OUT = './data/batch_data/'
df2.to_csv(PATH_OUT + 'csb_4.csv', index=False)